In [2]:
#features used in code
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
#from pydub import AudioSegment

In [15]:
#extracting features

def extract_feature(file_name,mfcc,chroma,mel):
    with soundfile.SoundFile(file_name)as sound_file:
        X=sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=40).T,axis=0)
            result = np.hstack((result,mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft,sr=sample_rate).T,axis=0)
            result=np.hstack((result,chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X,sr=sample_rate).T,axis=0)
            result=np.hstack((result,mel))
    return result

In [16]:
#define dataset to hold numbers and emotions available in the dataset

emotions={
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprised'
}

#define a list to hold emotion those we want

observed_emotions=['calm','happy','fearful','disgust']

In [17]:
#loading the data

def load_data(test_size=0.25):
    X,y=[],[]
    for file in glob.glob("PUT THE ADDRESS OF DATA FILE IN THIS SPACE IN FORMAT OF USING \\ IN PLACE OF \ AND ADDRESS ENDS WITH Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        #print(file)
        emotion=emotions[file_name.split("-")[0]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file,mfcc=True,chroma=True,mel=True)
        X.append(feature)
        y.append(emotion) 
    return train_test_split(np.array(X),y,test_size=test_size,random_state=77) 

In [18]:
X_train, X_test, y_train, y_test= load_data(test_size=0.25)

In [19]:
print((X_train.shape[0], X_test.shape[0]))


(1080, 360)


In [8]:
print(f'Features extracted: {X_train.shape[1]}')


Features extracted: 180


In [9]:
model = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, 
hidden_layer_sizes=(300,),learning_rate='adaptive' , max_iter=500)


In [10]:
model.fit(X_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [11]:
predictions = model.predict(X_test)


In [12]:
accuracy = accuracy_score(y_test, predictions) 
print("Accuracy: {:.2f}%".format(accuracy*100))


Accuracy: 100.00%


In [13]:
from sklearn.metrics import classification_report,confusion_matrix 
print(classification_report(y_test,predictions))


              precision    recall  f1-score   support

       happy       1.00      1.00      1.00       360

    accuracy                           1.00       360
   macro avg       1.00      1.00      1.00       360
weighted avg       1.00      1.00      1.00       360

